# Syntax

## Las score

Las score is a number between 0 and 1 that shows the proportion of equal arcs starting in the window.

In [1]:
from estnltk import Text
from estnltk.syntax.scoring import las_score
from estnltk.converters.conll_importer import conll_to_text, add_layer_from_conll


text = conll_to_text(file='a.conll', syntax_layer='conll_a')
add_layer_from_conll(file='a.conll', text=text, syntax_layer='conll_b')

Text(text='Milliseks kujuneb Riigikassa ja Ühispanga vahekord ? Minu arvates on Eesti pangandus tehnoloogiliselt maailma tasemel .')

Change the second layer a little bit.

In [2]:
text.conll_b[3].deprel = '@X'

text.conll_a[3].deprel != text.conll_b[3].deprel

True

Calculate las score.

In [3]:
las_score(layer_a=text.conll_a,
          layer_b=text.conll_b,
          start=None,
          end=None
          )

0.9375

Compare first 4 spans.

In [4]:
las_score(text.conll_a, text.conll_b, 0, 4)

0.75

Compare spans starting from the fifth.

In [5]:
las_score(text.conll_a, text.conll_b, 4)

1.0

## Tag sliding las scores

In [6]:
from estnltk.taggers.syntax.syntax_las_tagger import SyntaxLasTagger

tagger = SyntaxLasTagger('conll_a', 'conll_b', window=3)
tagger

SyntaxLasTagger(input_layers=('conll_a', 'conll_b'), output_layer=las, output_attributes=('deprel_sequence', 'score'), window=3)

In [7]:
tagger.tag(text)
text.las

Layer(name='las', attributes=('deprel_sequence', 'score'), spans=SL[ES[AS[Annotation(Milliseks, {'children': None, 'deprel': '@ADVL', 'deps': None, 'feats': 'rel|sg|tr', 'head': 2, 'id': 1, 'lemma': 'milline', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'P', 'xpostag': 'P'})]],
ES[AS[Annotation(Milliseks, {'children': None, 'deprel': '@ADVL', 'deps': None, 'feats': 'rel|sg|tr', 'head': 2, 'id': 1, 'lemma': 'milline', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'P', 'xpostag': 'P'})],
AS[Annotation(kujuneb, {'children': None, 'deprel': 'ROOT', 'deps': None, 'feats': 'indic|pres|ps3|sg', 'head': 0, 'id': 2, 'lemma': 'kujune', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'V', 'xpostag': 'V'})]],
ES[AS[Annotation(Milliseks, {'children': None, 'deprel': '@ADVL', 'deps': None, 'feats': 'rel|sg|tr', 'head': 2, 'id': 1, 'lemma': 'milline', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'P', 'xpostag': 'P'})],
AS[Annotation(kujuneb, {'children': None, 'deprel': 'ROOT', 'deps': None, 'feats': 'indic|pres|ps3|sg', 'head': 0, 'id': 2, 'lemma': 'kujune', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'V', 'xpostag': 'V'})],
AS[Annotation(Riigikassa, {'children': None, 'deprel': '@SUBJ', 'deps': None, 'feats': 'sg|nom', 'head': 2, 'id': 3, 'lemma': 'Riigi_kassa', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'S', 'xpostag': 'H'})]],
ES[AS[Annotation(kujuneb, {'children': None, 'deprel': 'ROOT', 'deps': None, 'feats': 'indic|pres|ps3|sg', 'head': 0, 'id': 2, 'lemma': 'kujune', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'V', 'xpostag': 'V'})],
AS[Annotation(Riigikassa, {'children': None, 'deprel': '@SUBJ', 'deps': None, 'feats': 'sg|nom', 'head': 2, 'id': 3, 'lemma': 'Riigi_kassa', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'S', 'xpostag': 'H'})],
AS[Annotation(ja, {'children': None, 'deprel': '@J', 'deps': None, 'feats': None, 'head': 6, 'id': 4, 'lemma': 'ja', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'J', 'xpostag': 'Jc'})]],
ES[AS[Annotation(Riigikassa, {'children': None, 'deprel': '@SUBJ', 'deps': None, 'feats': 'sg|nom', 'head': 2, 'id': 3, 'lemma': 'Riigi_kassa', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'S', 'xpostag': 'H'})],
AS[Annotation(ja, {'children': None, 'deprel': '@J', 'deps': None, 'feats': None, 'head': 6, 'id': 4, 'lemma': 'ja', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'J', 'xpostag': 'Jc'})],
AS[Annotation(Ühispanga, {'children': None, 'deprel': '@NN>', 'deps': None, 'feats': 'sg|gen', 'head': 6, 'id': 5, 'lemma': 'ühis_pank', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'S', 'xpostag': 'S'})]],
ES[AS[Annotation(ja, {'children': None, 'deprel': '@J', 'deps': None, 'feats': None, 'head': 6, 'id': 4, 'lemma': 'ja', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'J', 'xpostag': 'Jc'})],
AS[Annotation(Ühispanga, {'children': None, 'deprel': '@NN>', 'deps': None, 'feats': 'sg|gen', 'head': 6, 'id': 5, 'lemma': 'ühis_pank', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'S', 'xpostag': 'S'})],
AS[Annotation(vahekord, {'children': None, 'deprel': '@SUBJ', 'deps': None, 'feats': 'sg|nom', 'head': 2, 'id': 6, 'lemma': 'vahe_kord', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'S', 'xpostag': 'S'})]],
ES[AS[Annotation(Ühispanga, {'children': None, 'deprel': '@NN>', 'deps': None, 'feats': 'sg|gen', 'head': 6, 'id': 5, 'lemma': 'ühis_pank', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'S', 'xpostag': 'S'})],
AS[Annotation(vahekord, {'children': None, 'deprel': '@SUBJ', 'deps': None, 'feats': 'sg|nom', 'head': 2, 'id': 6, 'lemma': 'vahe_kord', 'misc': None, 'parent_deprel': None, 'parent_span': None, 'upostag': 'S', 'xpostag': 'S'})],
AS[Annotation(?,

In [8]:
text.las.meta['aggregate_deprel_sequences']

{('@ADVL',): [1.0, 1.0],
 ('@ADVL', '@ADVL'): [1.0],
 ('@ADVL', '@ADVL', 'ROOT'): [1.0],
 ('@ADVL', '@NN>', '@ADVL'): [1.0],
 ('@ADVL', '@Punc'): [1.0],
 ('@ADVL', 'ROOT'): [1.0],
 ('@ADVL', 'ROOT', '@NN>'): [1.0],
 ('@ADVL', 'ROOT', '@SUBJ'): [1.0],
 ('@J', '@NN>', '@SUBJ'): [0.6666666666666666],
 ('@NN>', '@ADVL', '@Punc'): [1.0],
 ('@NN>', '@SUBJ', '@ADVL'): [1.0],
 ('@NN>', '@SUBJ', '@Punc'): [1.0],
 ('@Punc',): [1.0, 1.0],
 ('@SUBJ', '@ADVL', '@NN>'): [1.0],
 ('@SUBJ', '@J', '@NN>'): [0.6666666666666666],
 ('@SUBJ', '@Punc'): [1.0],
 ('ROOT', '@NN>', '@SUBJ'): [1.0],
 ('ROOT', '@SUBJ', '@J'): [0.6666666666666666]}

## `VislTagger`

In [9]:
from estnltk.taggers import VislTagger

text = Text('Ise alles tee esimesel poolel , vaevu kolmekümnekolmene .')
text.tag_layer(['morph_extended'])

visl_tagger = VislTagger()
visl_tagger.tag(text)

text.visl

Layer(name='visl', attributes=('lemma', 'ending', 'partofspeech', 'feats', 'deprel', 'head'), spans=SL[AS[Annotation(Ise, {'deprel': '@ADVL', 'ending': '0', 'feats': 'pos det refl pl nom cap', 'head': '#1->3', 'lemma': 'ise', 'partofspeech': 'P'}), Annotation(Ise, {'deprel': '@ADVL', 'ending': '0', 'feats': 'pos det refl sg nom cap', 'head': '#1->3', 'lemma': 'ise', 'partofspeech': 'P'})],
AS[Annotation(alles, {'deprel': '@ADVL', 'ending': '0', 'feats': '_', 'head': '#2->3', 'lemma': 'alles', 'partofspeech': 'D'})],
AS[Annotation(tee, {'deprel': '@NN>', 'ending': '0', 'feats': 'com sg gen', 'head': '#3->0', 'lemma': 'tee', 'partofspeech': 'S'}), Annotation(tee, {'deprel': '@OBJ', 'ending': '0', 'feats': 'com sg gen', 'head': '#3->0', 'lemma': 'tee', 'partofspeech': 'S'})],
AS[Annotation(esimesel, {'deprel': '@AN>', 'ending': 'l', 'feats': 'ord sg ad l', 'head': '#4->5', 'lemma': 'esimene', 'partofspeech': 'N'})],
AS[Annotation(poolel, {'deprel': '@<NN @ADVL', 'ending': 'l', 'feats': 'com sg ad', 'head': '#5->3', 'lemma': 'pool', 'partofspeech': 'S'})],
AS[Annotation(,, {'deprel': '_', 'ending': '_', 'feats': 'Com', 'head': '#6->6', 'lemma': ',', 'partofspeech': 'Z'})],
AS[Annotation(vaevu, {'deprel': '@ADVL', 'ending': '0', 'feats': '_', 'head': '#7->3', 'lemma': 'vaevu', 'partofspeech': 'D'})],
AS[Annotation(kolmekümnekolmene, {'deprel': '@ADVL', 'ending': '0', 'feats': 'pos sg nom', 'head': '#8->5', 'lemma': 'kolme_kümne_kolmene', 'partofspeech': 'A'})],
AS[Annotation(., {'deprel': '_', 'ending': '_', 'feats': 'Fst CLB', 'head': '#9->9', 'lemma': '.', 'partofspeech': 'Z'})]])

## `ConllMorphTagger`

In [10]:
from estnltk.taggers import ConllMorphTagger

tagger = ConllMorphTagger(output_layer='conll_morph',  # default: 'conll_morph'
                          morph_extended_layer='morph_extended'  # default: 'morph_extended'
                          )
tagger

name,output layer,output attributes,input layers
ConllMorphTagger,conll_morph,"('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc')","('morph_extended',)"


In [11]:
tagger.tag(text)

text.conll_morph

Layer(name='conll_morph', attributes=('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[AS[Annotation(Ise, {'deprel': '_', 'deps': '_', 'feats': 'pos|det|refl|pl|nom', 'form': 'Ise', 'head': '_', 'id': '1', 'lemma': 'ise', 'misc': '_', 'upostag': 'P', 'xpostag': 'P'})],
AS[Annotation(alles, {'deprel': '_', 'deps': '_', 'feats': '_', 'form': 'alles', 'head': '_', 'id': '2', 'lemma': 'alles', 'misc': '_', 'upostag': 'D', 'xpostag': 'D'})],
AS[Annotation(tee, {'deprel': '_', 'deps': '_', 'feats': 'sg|gen', 'form': 'tee', 'head': '_', 'id': '3', 'lemma': 'tee', 'misc': '_', 'upostag': 'S', 'xpostag': 'S'})],
AS[Annotation(esimesel, {'deprel': '_', 'deps': '_', 'feats': 'ord|sg|ad|l', 'form': 'esimesel', 'head': '_', 'id': '4', 'lemma': 'esimene', 'misc': '_', 'upostag': 'N', 'xpostag': 'A'})],
AS[Annotation(poolel, {'deprel': '_', 'deps': '_', 'feats': 'sg|ad', 'form': 'poolel', 'head': '_', 'id': '5', 'lemma': 'pool', 'misc': '_', 'upostag': 'S', 'xpostag': 'S'})],
AS[Annotation(,, {'deprel': '_', 'deps': '_', 'feats': 'Com', 'form': ',', 'head': '_', 'id': '6', 'lemma': ',', 'misc': '_', 'upostag': 'Z', 'xpostag': 'Z'})],
AS[Annotation(vaevu, {'deprel': '_', 'deps': '_', 'feats': '_', 'form': 'vaevu', 'head': '_', 'id': '7', 'lemma': 'vaevu', 'misc': '_', 'upostag': 'D', 'xpostag': 'D'})],
AS[Annotation(kolmekümnekolmene, {'deprel': '_', 'deps': '_', 'feats': 'sg|nom', 'form': 'kolmekümnekolmene', 'head': '_', 'id': '8', 'lemma': 'kolme_kümne_kolmene', 'misc': '_', 'upostag': 'A', 'xpostag': 'A'})],
AS[Annotation(., {'deprel': '_', 'deps': '_', 'feats': 'Fst', 'form': '.', 'head': '_', 'id': '9', 'lemma': '.', 'misc': '_', 'upostag': 'Z', 'xpostag': 'Z'})]])

Convert conll_morph layer to CoNLL-format string.

In [12]:
from estnltk.taggers.syntax.conll_morph_to_str import *

print(conll_to_str(text))

1	Ise	ise	P	P	pos|det|refl|pl|nom	_	_	_	_	
2	alles	alles	D	D	_	_	_	_	_	
3	tee	tee	S	S	sg|gen	_	_	_	_	
4	esimesel	esimene	N	A	ord|sg|ad|l	_	_	_	_	
5	poolel	pool	S	S	sg|ad	_	_	_	_	
6	,	,	Z	Z	Com	_	_	_	_	
7	vaevu	vaevu	D	D	_	_	_	_	_	
8	kolmekümnekolmene	kolme_kümne_kolmene	A	A	sg|nom	_	_	_	_	
9	.	.	Z	Z	Fst	_	_	_	_	




# Maltparser

Parse CoNLL-format string with maltparser.

In [13]:
from estnltk.taggers.syntax.maltparser import MaltParser

parser = MaltParser()
initial_output = parser.parse_text(text, return_type='conll')
print( '\n'.join( initial_output) )

1	Ise	ise	P	P	pos|det|refl|pl|nom	0	ROOT	_	_
2	alles	alles	D	D	_	3	@DN>	_	_
3	tee	tee	S	S	sg|gen	5	@NN>	_	_
4	esimesel	esimene	N	A	ord|sg|ad|l	5	@AN>	_	_
5	poolel	pool	S	S	sg|ad	1	@ADVL	_	_
6	,	,	Z	Z	Com	5	@Punc	_	_
7	vaevu	vaevu	D	D	_	8	@ADVL	_	_
8	kolmekümnekolmene	kolme_kümne_kolmene	A	A	sg|nom	1	@PRD	_	_
9	.	.	Z	Z	Fst	8	@Punc	_	_

